<a href="https://colab.research.google.com/github/victorog17/soulcode_aulas_apache_beam/blob/main/Apache_Beam_002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**DOCUMENTAÇÃO**

https://cloud.google.com/dataflow/docs/reference/pipeline-options#python

**INSTALANDO APACHE BEAM GCP**

In [ ]:
#pip install apache-beam[interactive]

In [ ]:
#pip install apache-beam[gcp]

**BIBLIOTECAS**

In [ ]:
import apache_beam as beam
import os

**SALVANDO EM CLOUD E CRIANDO DATAFLOW**

In [ ]:
import apache_beam as beam
import os
from apache_beam.options.pipeline_options import PipelineOptions

pipeline_options = {
    'project': 'ingestao-apache-beam-victor', #nome do projeto (Project ID)
    'runner': 'DataFlowRunner', # executor do projeto
    'region': 'southamerica-east1',
    'staging_location': 'gs://eng_gcp_dados_victor_soulcode001/temp', # área de preparação, o que de arquivo aquela VM precisa instalar para rodar aqui (tipo APIs que serão usadas)
    'temp_location': 'gs://eng_gcp_dados_victor_soulcode001/temp', # o local onde ele irá processar
    'template_location': 'gs://eng_gcp_dados_victor_soulcode001/template/template_arq_batch' # onde ficará o template/destino
}

serviceAccount = '/content/drive/MyDrive/Soul_Code_Academy/repositorio_apache_beam/ingestao-apache-beam-victor-0ac6fa7d11ea.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

pipeline_options = PipelineOptions.from_dictionary(pipeline_options)
p1 = beam.Pipeline(options=pipeline_options)

atrasados_tempo = (
    p1
    |'1.Extrair os dados' >> beam.io.ReadFromText('gs://eng_gcp_dados_victor_soulcode001/entrada/voos_sample.csv', skip_header_lines= 0)
    |'1.Separador' >> beam.Map(lambda record: record.split(','))
    |'1.Aeroportos atrasados' >> beam.Filter(lambda record: int(record[8]) > 0)
    |'1.Agregação de colunas' >> beam.Map(lambda record: (record[3], int(record[8])))
    |'1.Combinar os dados' >> beam.CombinePerKey(sum)
    #|'1.Gravar resultado' >> beam.io.WriteToText('/content/drive/MyDrive/Soul_Code_Academy/repositorio_apache_beam/resultato_voos_atrasados.txt')
)

atrasos_qtd = (
    p1
    |'2.Extrair os dados' >> beam.io.ReadFromText('gs://eng_gcp_dados_victor_soulcode001/entrada/voos_sample.csv', skip_header_lines= 0)
    |'2.Separador' >> beam.Map(lambda record: record.split(','))
    |'2.Aeroportos atrasados' >> beam.Filter(lambda record: int(record[8]) > 0)
    |'2.Agregação de colunas' >> beam.Map(lambda record: (record[3], int(record[8])))
    |'2.Quantidade de atrasos' >> beam.combiners.Count.PerKey()
    #|'2.Gravar resultado' >> beam.io.WriteToText('/content/drive/MyDrive/Soul_Code_Academy/repositorio_apache_beam/resultato_voos_atrasados.txt')
)

tabela_final = (
    {'3.Qtd_Atrasos': atrasos_qtd, 'Tempo_Atrasos': atrasados_tempo}
    |'3.Agrupamento das Pipes' >> beam.CoGroupByKey()
    #|'3.Imprimir resultado' >> beam.Map(print)
    #|'3.Saída para GCP' >> beam.io.WriteToText('gs://eng_gcp_dados_victor_soulcode001/teste/TESTE.csv')
)
p1.run()

<DataflowPipelineResult None at 0x7f604037ec10>